In [2]:
%pip install git+https://github.com/keras-team/keras-cv

  Cloning https://github.com/keras-team/keras-cv to /tmp/pip-req-build-u_7lvq1x
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-cv /tmp/pip-req-build-u_7lvq1x
  Resolved https://github.com/keras-team/keras-cv to commit 94b0a551d03ee9de3d81663e2c1a680cb113f7f1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 5.9 MB/s eta 0:00:0000:01
  Created wheel for keras-cv: filename=keras_cv-0.10.0-py3-none-any.whl size=975496 sha256=b3ddddd13cd867a16ffdb04414056c04a1f554d28e13afe0c2494c66cdb66395
  Stored in directory: /tmp/pip-ephem-wheel-cache-s2a6wgcb/wheels/22/4a/22/95891344ab2777b97e94694fd4a187de0b50b831e9027b0f56
Successfully built keras-cv
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  At

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_cv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

2025-04-16 07:16:14.570566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744787774.841564      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744787774.923714      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Load the CSV file
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'  # Change this to your CSV file path
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'  # Change this to your image folder path
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column

In [5]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)

In [6]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 26284 validated image filenames belonging to 7 classes.
Found 6570 validated image filenames belonging to 7 classes.
Found 14081 validated image filenames belonging to 7 classes.


In [8]:
from tensorflow.keras import layers, models, backend as K

def shuffle_unit(inputs, out_channels, bottleneck_ratio=1, stride=1):
    in_channels = inputs.shape[-1]

    if stride == 1:
        x1 = layers.Lambda(lambda z: z[:, :, :, :in_channels // 2])(inputs)
        x2 = layers.Lambda(lambda z: z[:, :, :, in_channels // 2:])(inputs)
        shortcut = x1
        x = x2
    else:
        shortcut = inputs
        x = inputs

    x = layers.Conv2D(out_channels // 2, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.DepthwiseConv2D(3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(out_channels // 2, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    if stride == 1:
        out = layers.Concatenate()([shortcut, x])
    else:
        shortcut = layers.AveragePooling2D(pool_size=3, strides=2, padding='same')(shortcut)
        out = layers.Concatenate()([shortcut, x])

    return out

def ShuffleNetV2(input_shape=(96, 96, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(24, 3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Shuffle units
    for _ in range(3):
        x = shuffle_unit(x, 48)
    for _ in range(7):
        x = shuffle_unit(x, 96, stride=2)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, x)


model = ShuffleNetV2(input_shape=(96, 96, 3), num_classes=len(label_encoder.classes_))
model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])


2025-04-16 07:18:40.772755: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


822/822 ━━━━━━━━━━━━━━━━━━━━ 414s 477ms/step - accuracy: 0.2190 - loss: 1.9409 - val_accuracy: 0.3504 - val_loss: 1.6519
Epoch 2/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 313s 379ms/step - accuracy: 0.3428 - loss: 1.6417 - val_accuracy: 0.4126 - val_loss: 1.4850
Epoch 3/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 290s 350ms/step - accuracy: 0.4119 - loss: 1.4933 - val_accuracy: 0.4324 - val_loss: 1.4030
Epoch 4/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 288s 349ms/step - accuracy: 0.4399 - loss: 1.4117 - val_accuracy: 0.4849 - val_loss: 1.3089
Epoch 5/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 292s 354ms/step - accuracy: 0.4696 - loss: 1.3503 - val_accuracy: 0.4986 - val_loss: 1.2667
Epoch 6/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 296s 358ms/step - accuracy: 0.4842 - loss: 1.2950 - val_accuracy: 0.5120 - val_loss: 1.2476
Epoch 7/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 290s 351ms/step - accuracy: 0.4998 - loss: 1.2572 - val_accuracy: 0.5323 - val_loss: 1.2007
Epoch 8/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 285s 345ms/step - accuracy: 0.5168 - loss: 1.23

In [10]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [12]:
plot_model_training_curve(history)

In [13]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

  1/441 ━━━━━━━━━━━━━━━━━━━━ 1:35 216ms/step - accuracy: 0.3750 - loss: 1.7442

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



441/441 ━━━━━━━━━━━━━━━━━━━━ 156s 353ms/step - accuracy: 0.5506 - loss: 1.2337

Test Accuracy: 55.44%


In [14]:
model.save('/kaggle/working/shufflenet_model.keras') 
model.save_weights('/kaggle/working/shufflenet_model_weights.weights.h5') 